In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath("../../"), "src"))
sys.path.append(os.path.join(os.path.abspath("."), "src"))
os.environ['HF_HOME'] = "/data/workspace/alexww14/2025-cv/notebooks/milstone2/cache"
os.environ['TRANSFORMERS_CACHE'] = "/data/workspace/alexww14/2025-cv/notebooks/milstone2/cache"

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from transformers import BertForSequenceClassification, BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bertopic import BERTopic
import torch

from scipy.special import softmax
from model_evaluation import get_model_metrics

In [ ]:
# with open('/data/workspace/dataset/stacking_predictions.pkl', 'rb') as f:
#     df = pickle.load(f)

In [ ]:
df = pd.read_parquet('/data/workspace/dataset/sampled-dataset/raw/sample-small.parquet')
sample_df = df.sample(100, random_state=123)

In [ ]:
# def create_prediction_label(row):

#     if (row['target_1'] == 'malicious') & (row['predicted_target'] == 'malicious'):
#         return 'true_positive'
    
#     if (row['target_1'] == 'malicious') & (row['predicted_target'] == 'benign'):
#         return 'false_negative'
    
#     if (row['target_1'] == 'benign') & (row['predicted_target'] == 'malicious'):
#         return 'false_positive'
    
#     if (row['target_1'] == 'benign') & (row['predicted_target'] == 'benign'):
#         return 'true_negative'

In [ ]:
# df['prediction_label'] = df.apply(lambda row: create_prediction_label(row), axis=1)


### Sentiment analysis

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
# text = "hello find enclose invoice payment examination access document document protect view hesitate contact question thank cal albright msw executive executive director kamloops aboriginal friendship society palm phone ext fax mail"
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
# scores = output[0][0].detach().numpy()
# scores = softmax(scores)

In [ ]:
def get_sentiment(model, tokenizer, text):
    try:
        encoded_input = tokenizer(text, return_tensors='pt')
    except RuntimeError:
        return 'str too long'
    
    try:
        output = model(**encoded_input)
    except RuntimeError:
        return 'str too long'
    
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    pos = np.where(scores == max(scores))[0][0]

    if pos == 0:
        return 'negative'
    if pos == 1:
        return 'neutral'
    else:
        return 'positive'


In [ ]:
toy_sentiment = sample_df[['text_preprocessed', 'target_1']]

In [ ]:
toy_sentiment['sentiment'] = toy_sentiment['text_preprocessed'].apply(lambda x: get_sentiment(model, tokenizer, x))

In [ ]:
toy_sentiment.to_csv('/data/workspace/alexww14/2025-cv/notebooks/milestone2/BERT_sentiment_results.csv')

In [ ]:
toy_sentiment = pd.read_csv('/data/workspace/alexww14/2025-cv/notebooks/milestone2/BERT_sentiment_results.csv')

In [ ]:
neutral_count = len(toy_sentiment[toy_sentiment['sentiment']=='neutral'])
pos_count = len(toy_sentiment[toy_sentiment['sentiment']=='positive'])
neg_count = len(toy_sentiment[toy_sentiment['sentiment']=='negative'])
parsing_error_count = len(toy_sentiment[toy_sentiment['sentiment']=='str too long'])

In [ ]:
sentiments_table = {
    'Positive': pos_count,
    'Neutral': neutral_count,
    'Negative': neg_count,
    'Parsing Error': parsing_error_count
}

In [ ]:
pd.DataFrame([sentiments_table])

### BERTopic

In [ ]:
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

In [ ]:
def get_topic(model, text):
    topic, prob = model.transform(text)

    return model.topic_labels_[topic[0]]

In [ ]:
toy_topic = sample_df[['text_preprocessed', 'target_1']]

In [ ]:
toy_topic['topic_MaartenGr'] = toy_topic['text_preprocessed'].apply(lambda x: get_topic(topic_model, x) if x is not None else " ")

In [ ]:
toy_topic.to_csv('/data/workspace/alexww14/2025-cv/notebooks/milestone2/BERT_topic_results.csv')

In [ ]:
topic_count = pd.DataFrame(toy_topic[['topic_MaartenGr']].value_counts())
topic_count.reset_index(inplace=True)
topic_count['label_num'] = topic_count['topic_MaartenGr'].str.split("_").str[0]

In [ ]:
topic_count.to_csv('/data/workspace/alexww14/2025-cv/notebooks/milestone2/topic_count.csv')

In [ ]:
import altair as alt 

email_count_per_topic_chart = alt.Chart(topic_count).mark_bar().encode(
    x=alt.X('label_num:N', sort='-y', title='Topic Label Number'),
    y='count'
).properties(
    title='Email Count per Topic'
)

email_count_per_topic_chart

In [ ]:
email_count_per_topic_chart.save('email_count_per_topic_chart.png')

In [ ]:
topic_count = pd.DataFrame(toy_topic[['topic_MaartenGr']].value_counts())
topic_count.reset_index(inplace=True)
topic_count['label_num'] = topic_count['topic_MaartenGr'].str.split("_").str[0]

In [ ]:
topic_count.to_csv('/data/workspace/alexww14/2025-cv/notebooks/milestone2/topic_count.csv')

### ElSlay/BERT-Phishing-Email-Model

In [ ]:
model_name = 'ElSlay/BERT-Phishing-Email-Model'
 
# Load the pre-trained model and tokenizer
model_elslay = BertForSequenceClassification.from_pretrained(model_name, cache_dir='/data/workspace/alexww14/2025-cv/notebooks/milstone2/cache')
tokenizer_elslay = BertTokenizer.from_pretrained(model_name)

# Ensure the model is in evaluation mode
model_elslay.eval()

In [ ]:
def BERT_prediction(model, tokenizer, email_text):

    if not isinstance(email_text, str):
        email_text = str(email_text)

    # Tokenize and preprocess the input text
    inputs = tokenizer(email_text, return_tensors="pt", truncation=True, padding='max_length', max_length=512)

    # Make the prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    # Interpret the prediction
    result = "malicious" if predictions.item() == 1 else "benign"
    
    return result

In [ ]:
toy_classification = sample_df[['text_clean', 'target_1']]

In [ ]:
toy_classification['prediction_elshay'] = toy_classification['text_clean'].apply(lambda x: BERT_prediction(model_elslay, tokenizer_elslay,x))

In [ ]:
toy_classification['prediction_elshay']

In [ ]:
result_elshay = get_model_metrics(toy_classification['target_1'], toy_classification['prediction_elshay'])

In [ ]:
result_elshay

In [ ]:
with open('/data/workspace/alexww14/2025-cv/results/BERT/result_elshay.pkl', 'wb') as f:
    pickle.dump(result_elshay, f)

In [ ]:
with open("/data/workspace/alexww14/2025-cv/results/BERT/result_elshay.pkl", 'rb') as f:
    result_elshay = pickle.load(f)

In [ ]:
pd.DataFrame([result_elshay]).round(3)

#### ealvaradob/bert-finetuned-phishing

In [ ]:
model_name = "ealvaradob/bert-finetuned-phishing"
 
# Load the pre-trained model and tokenizer
model_ealvaradob = BertForSequenceClassification.from_pretrained(model_name)
tokenizer_ealvaradob = BertTokenizer.from_pretrained(model_name)

# Ensure the model is in evaluation mode
model_ealvaradob.eval()

In [ ]:
toy_classification['prediction_ealvaradob'] = toy_classification['text_clean'].apply(lambda x: BERT_prediction(model_ealvaradob, tokenizer_ealvaradob,x))

In [ ]:
result_ealvaradob = get_model_metrics(toy_classification['target_1'], toy_classification['prediction_ealvaradob'])

In [ ]:
with open('result_ealvaradob.pkl', 'wb') as f:
    pickle.dump(result_ealvaradob, f)

In [ ]:
with open('result_ealvaradob.pkl', 'rb') as f:
    result_ealvaradob = pickle.load(f)

In [ ]:
pd.DataFrame([result_ealvaradob])

In [ ]:
results_df = pd.DataFrame([result_elshay,result_ealvaradob], index=['elslay', 'ealvaradob']).round(3)

In [ ]:
tidy_df = pd.DataFrame({
        'Model': results_df.index,
        'Precision': results_df['precision'],
        'Recall': results_df['recall'],
        'F1-score': results_df['f1-score'],
        'False Benign Rate / FNR': results_df['false_benign_rate'],
        'False Malicious Rate / FPR': results_df['false_malicious_rate']
    })
tidy_df